# 智能旅行助手 - ReAct 智能体演示

本 Notebook 演示如何从零构建一个基于 ReAct 范式的智能旅行助手。

## 功能特性
- 🌤️ 查询实时天气
- 🏛️ 推荐旅游景点
- 🤖 自主规划任务执行流程

## 1. 环境准备与依赖安装

In [1]:
# 安装必要的依赖包
!pip install requests tavily-python openai python-dotenv -q

## 2. 导入依赖与加载 API Key

In [2]:
import os
import re
import requests
from openai import OpenAI
from tavily import TavilyClient
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv

# 验证 API Key 是否加载成功
tavily_key = os.getenv('TAVILY_API_KEY')
print(f"✅ Tavily API Key 已加载" if tavily_key else "❌ Tavily API Key 未找到")

# 使用智谱AI (可以根据需要更改为其他 LLM)
API_KEY = os.getenv('ZHIPU_API_KEY')
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"
MODEL_ID = "glm-4-flash"

print(f"✅ LLM API Key 已加载" if API_KEY else "❌ LLM API Key 未找到")

✅ Tavily API Key 已加载
✅ LLM API Key 已加载


## 3. 实现工具函数

### 3.1 天气查询工具

In [3]:
def get_weather(city: str) -> str:
    """
    通过调用 wttr.in API 查询真实的天气信息。
    
    参数:
        city: 城市名称
    
    返回:
        天气信息的字符串描述
    """
    # API端点，我们请求JSON格式的数据
    url = f"https://wttr.in/{city}?format=j1"
    
    try:
        # 发起网络请求
        response = requests.get(url)
        # 检查响应状态码是否为200 (成功)
        response.raise_for_status() 
        # 解析返回的JSON数据
        data = response.json()
        
        # 提取当前天气状况
        current_condition = data['current_condition'][0]
        weather_desc = current_condition['weatherDesc'][0]['value']
        temp_c = current_condition['temp_C']
        
        # 格式化成自然语言返回
        return f"{city}当前天气:{weather_desc}，气温{temp_c}摄氏度"
        
    except requests.exceptions.RequestException as e:
        # 处理网络错误
        return f"错误:查询天气时遇到网络问题 - {e}"
    except (KeyError, IndexError) as e:
        # 处理数据解析错误
        return f"错误:解析天气数据失败，可能是城市名称无效 - {e}"

# 测试天气查询功能
print("🔧 测试天气查询工具:")
print(get_weather("北京"))

🔧 测试天气查询工具:
北京当前天气:Clear，气温-1摄氏度


### 3.2 景点推荐工具

In [4]:
def get_attraction(city: str, weather: str) -> str:
    """
    根据城市和天气，使用Tavily Search API搜索并返回优化后的景点推荐。
    
    参数:
        city: 城市名称
        weather: 天气描述
    
    返回:
        景点推荐的字符串描述
    """
    # 1. 从环境变量中读取API密钥
    api_key = os.environ.get("TAVILY_API_KEY")
    if not api_key:
        return "错误:未配置TAVILY_API_KEY环境变量。"

    # 2. 初始化Tavily客户端
    tavily = TavilyClient(api_key=api_key)
    
    # 3. 构造一个精确的查询
    query = f"'{city}' 在'{weather}'天气下最值得去的旅游景点推荐及理由"
    
    try:
        # 4. 调用API，include_answer=True会返回一个综合性的回答
        response = tavily.search(query=query, search_depth="basic", include_answer=True)
        
        # 5. Tavily返回的结果已经非常干净，可以直接使用
        # response['answer'] 是一个基于所有搜索结果的总结性回答
        if response.get("answer"):
            return response["answer"]
        
        # 如果没有综合性回答，则格式化原始结果
        formatted_results = []
        for result in response.get("results", []):
            formatted_results.append(f"- {result['title']}: {result['content']}")
        
        if not formatted_results:
             return "抱歉，没有找到相关的旅游景点推荐。"

        return "根据搜索，为您找到以下信息:\n" + "\n".join(formatted_results)

    except Exception as e:
        return f"错误:执行Tavily搜索时出现问题 - {e}"

# 测试景点推荐功能
print("🔧 测试景点推荐工具:")
print(get_attraction("北京", "Sunny"))

🔧 测试景点推荐工具:
Under sunny weather, visit the Summer Palace and the Great Wall for beautiful views and historical significance. Both are popular and well-known attractions in Beijing.


## 4. 实现 LLM 客户端

In [5]:
class OpenAICompatibleClient:
    """
    一个用于调用任何兼容OpenAI接口的LLM服务的客户端。
    """
    def __init__(self, model: str, api_key: str, base_url: str):
        self.model = model
        self.client = OpenAI(api_key=api_key, base_url=base_url)

    def generate(self, prompt: str, system_prompt: str) -> str:
        """调用LLM API来生成回应。"""
        print("🤖 正在调用大语言模型...")
        try:
            messages = [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': prompt}
            ]
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                stream=False
            )
            answer = response.choices[0].message.content
            print("✅ 大语言模型响应成功。")
            return answer
        except Exception as e:
            print(f"❌ 调用LLM API时发生错误: {e}")
            return "错误:调用语言模型服务时出错。"

# 初始化 LLM 客户端
llm = OpenAICompatibleClient(
    model=MODEL_ID,
    api_key=API_KEY,
    base_url=BASE_URL
)

print("✅ LLM 客户端初始化完成")

✅ LLM 客户端初始化完成


## 5. 定义智能体系统提示词

In [6]:
AGENT_SYSTEM_PROMPT = """
你是一个智能旅行助手。你的任务是分析用户的请求，并使用可用工具一步步地解决问题。

# 可用工具:
- `get_weather(city: str)`: 查询指定城市的实时天气。
- `get_attraction(city: str, weather: str)`: 根据城市和天气搜索推荐的旅游景点。

# 行动格式:
你的回答必须严格遵循以下格式。首先是你的思考过程，然后是你要执行的具体行动，每次回复只输出一对Thought-Action：
Thought: [这里是你的思考过程和下一步计划]
Action: [这里是你要调用的工具，格式为 function_name(arg_name="arg_value")]

# 任务完成:
当你收集到足够的信息，能够回答用户的最终问题时，你必须在`Action:`字段后使用 `finish(answer="...")` 来输出最终答案。

请开始吧！
"""

print("✅ 系统提示词已定义")

✅ 系统提示词已定义


## 6. 实现智能体主循环

In [7]:
def run_agent(user_prompt: str, max_iterations: int = 5):
    """
    运行智能体主循环，处理用户请求。
    
    参数:
        user_prompt: 用户输入的请求
        max_iterations: 最大循环次数，防止无限循环
    """
    # 配置工具字典
    available_tools = {
        "get_weather": get_weather,
        "get_attraction": get_attraction,
    }
    
    # 初始化对话历史
    prompt_history = [f"用户请求: {user_prompt}"]
    
    print("="*60)
    print(f"👤 用户输入: {user_prompt}")
    print("="*60)
    print()
    
    # 运行主循环
    for i in range(max_iterations):
        print(f"\n{'='*60}")
        print(f"🔄 循环 {i+1}/{max_iterations}")
        print("="*60)
        
        # 1. 构建完整的 Prompt
        full_prompt = "\n".join(prompt_history)
        
        # 2. 调用 LLM 进行思考
        llm_output = llm.generate(full_prompt, system_prompt=AGENT_SYSTEM_PROMPT)
        
        # 模型可能会输出多余的Thought-Action，需要截断
        match = re.search(r'(Thought:.*?Action:.*?)(?=\n\s*(?:Thought:|Action:|Observation:)|\Z)', llm_output, re.DOTALL)
        if match:
            truncated = match.group(1).strip()
            if truncated != llm_output.strip():
                llm_output = truncated
                print("⚠️  已截断多余的 Thought-Action 对")
        
        print(f"\n💭 模型输出:\n{llm_output}\n")
        prompt_history.append(llm_output)
        
        # 3. 解析并执行行动
        action_match = re.search(r"Action:(.*)", llm_output, re.DOTALL)
        if not action_match:
            print("❌ 解析错误:模型输出中未找到 Action。")
            break
        
        action_str = action_match.group(1).strip()
        
        # 检查是否完成任务
        if action_str.startswith("finish"):
            final_match = re.search(r'finish\(answer="(.*)"\)', action_str)
            if final_match:
                final_answer = final_match.group(1)
                print("="*60)
                print("🎉 任务完成！")
                print("="*60)
                print(f"\n📝 最终答案:\n{final_answer}\n")
                print("="*60)
                return final_answer
            else:
                print("❌ 解析错误: finish 函数格式不正确")
                break
        
        # 执行工具调用
        try:
            tool_name = re.search(r"(\w+)\(", action_str).group(1)
            args_str = re.search(r"\((.*)\)", action_str).group(1)
            kwargs = dict(re.findall(r'(\w+)="([^"]*)"', args_str))
            
            if tool_name in available_tools:
                print(f"🔧 执行工具: {tool_name}({', '.join([f'{k}="{v}"' for k, v in kwargs.items()])})")
                observation = available_tools[tool_name](**kwargs)
            else:
                observation = f"错误:未定义的工具 '{tool_name}'"
        except Exception as e:
            observation = f"错误:解析或执行工具时出错 - {e}"
        
        # 4. 记录观察结果
        observation_str = f"Observation: {observation}"
        print(f"\n👁️  {observation_str}\n")
        prompt_history.append(observation_str)
    
    print("\n⚠️  达到最大循环次数，任务未完成。")
    return None

print("✅ 智能体主循环函数已定义")

✅ 智能体主循环函数已定义


## 7. 运行智能体演示

### 示例 1: 查询天气并推荐景点

In [8]:
# 示例 1: 北京旅游
user_request_1 = "你好，请帮我查询一下今天北京的天气，然后根据天气推荐一个合适的旅游景点。"
run_agent(user_request_1)

👤 用户输入: 你好，请帮我查询一下今天北京的天气，然后根据天气推荐一个合适的旅游景点。


🔄 循环 1/5
🤖 正在调用大语言模型...
✅ 大语言模型响应成功。

💭 模型输出:
Thought: 首先需要查询北京的实时天气，然后根据天气情况推荐一个合适的旅游景点。
Action: get_weather(city="北京")

🔧 执行工具: get_weather(city="北京")

👁️  Observation: 北京当前天气:Clear，气温-1摄氏度


🔄 循环 2/5
🤖 正在调用大语言模型...
✅ 大语言模型响应成功。

💭 模型输出:
Thought: 北京今天是晴天，我可以推荐一些适合晴天的旅游景点。
Action: get_attraction(city="北京", weather="Clear")

🔧 执行工具: get_attraction(city="北京", weather="Clear")

👁️  Observation: In clear weather, the Forbidden City and Tiananmen Square are top Beijing attractions. The Great Wall at Mutianyu is also highly recommended. These sites offer historical insights and scenic views.


🔄 循环 3/5
🤖 正在调用大语言模型...
✅ 大语言模型响应成功。

💭 模型输出:
Thought: 我已经查询到了北京的天气，并且知道今天是晴天。现在我将根据这个天气情况推荐一些旅游景点。
Action: finish(answer="在北京晴天的情况下，我推荐您去故宫、天安门广场或者慕田峪长城。这些地方不仅能够让您了解历史，还能欣赏到美丽的风景。")

🎉 任务完成！

📝 最终答案:
在北京晴天的情况下，我推荐您去故宫、天安门广场或者慕田峪长城。这些地方不仅能够让您了解历史，还能欣赏到美丽的风景。



'在北京晴天的情况下，我推荐您去故宫、天安门广场或者慕田峪长城。这些地方不仅能够让您了解历史，还能欣赏到美丽的风景。'

### 示例 2: 上海旅游规划

In [ ]:
# 示例 2: 上海旅游
user_request_2 = "我想去上海旅游，帮我查一下上海的天气，并推荐适合的景点。"
run_agent(user_request_2)

### 示例 3: 自定义查询

In [ ]:
# 示例 3: 自定义查询 - 你可以修改这里的内容
custom_request = "帮我查询杭州的天气，如果天气好的话推荐一些户外景点。"
run_agent(custom_request)

## 8. 总结

### 🎯 核心概念

1. **ReAct 范式**: Thought（思考）→ Action（行动）→ Observation（观察）
2. **工具调用**: 智能体能够自主决定何时调用哪个工具
3. **循环执行**: 通过多轮交互逐步完成复杂任务

### 🔧 技术要点

- 使用 `wttr.in` API 获取实时天气数据
- 使用 `Tavily` 进行智能搜索和景点推荐
- 使用 OpenAI 兼容接口调用大语言模型
- 正则表达式解析 LLM 输出的结构化信息

### 🚀 扩展方向

1. 添加更多工具（酒店预订、交通查询等）
2. 优化提示词工程，提高任务完成率
3. 添加记忆机制，支持多轮对话
4. 实现错误重试和异常处理机制